# Enhanced Weekly Features Computation Pipeline

This notebook demonstrates the enhanced weekly features computation process with comprehensive ETF technical analysis.

## Enhanced Pipeline Overview

The enhanced orchestrator follows this structure for weekly features:
1. **Load raw stock and ETF data** using `load_stock_universe()` and `load_etf_universe()`
2. **Load enhanced sector mappings** with correlation-validated sector/subsector assignments
3. **Enhanced weekly processing** - single parallel job combining:
   - Weekly resampling (Friday bars)
   - Stock technical features (RSI, MACD, trend, distance)
   - ETF technical features (RSI, MACD, 3-day EMA of MACD slope) for:
     - SPY (market cap-weighted) & RSP (market equal-weighted)
     - Sector ETFs (cap-weighted) & Equal-weight sector ETFs 
     - Subsector ETFs (22 specialized ETFs like SMH, IBB, XOP)
   - Enhanced relative strength analysis
   - Weekly prefix addition and data type optimization

## Enhanced Features Generated (~150+ per symbol)
- **Stock Technical**: RSI, MACD, trend analysis, distance to MAs
- **ETF Technical**: RSI, MACD, MACD slope EMA3 for market/sector/subsector ETFs
- **Enhanced Relative Strength**: vs SPY, RSP, sectors, equal-weight sectors, subsectors
- **Cross-benchmark Analysis**: Sector vs market strength comparisons

## Performance Optimization
- **Single integrated parallel job** for maximum efficiency
- **Enhanced sector mappings** with correlation validation
- **Memory optimized** with `float32` types and efficient data structures
- **Comprehensive validation** with data quality assessment and visualization dashboard

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from typing import Dict, Optional, List, Tuple
from joblib import Parallel, delayed
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Add project src to path
project_root = os.path.abspath('..')
if os.path.exists(os.path.join(project_root, 'src')):
    sys.path.insert(0, os.path.join(project_root, 'src'))
else:
    # Try parent directory
    parent_dir = os.path.dirname(project_root)
    if os.path.exists(os.path.join(parent_dir, 'src')):
        sys.path.insert(0, os.path.join(parent_dir, 'src'))

print(f"📁 Working directory: {os.getcwd()}")
print(f"🔍 Project root: {project_root}")
print(f"🐍 Python path: {sys.path[:3]}...")

📁 Working directory: /mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard/notebooks
🔍 Project root: /mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard
🐍 Python path: ['/mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard/src', '/mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard/notebooks/src', '/mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/.local/share/mamba/envs/stocks_predictor/lib/python313.zip']...


In [2]:
# Import existing functions
# Import feature functions
from features.trend import add_trend_features, add_rsi_features, add_macd_features
from features.distance import add_distance_to_ma_features
from features.weekly import add_weekly_features_to_daily, _resample_to_weekly, _compute_weekly_features_single_symbol
from features.relstrength import _compute_relative_strength_block

# Import data loading functions
from data.loader import load_stock_universe, load_etf_universe

# Import daily feature processing functions (from orchestrator)
from features.assemble import assemble_indicators_from_wide
from features.volatility import add_multiscale_vol_regime
# from features.hurst import add_hurst_features
from features.range_breakout import add_range_breakout_features
from features.volume import add_volume_features, add_volume_shock_features
from features.ohlc_adjustment import adjust_ohlc_to_adjclose

print("📈 Loading stock universe data...")


📈 Loading stock universe data...


In [3]:
stock_data_dict = load_stock_universe()                                                                                     
                                                                                                                                  
if stock_data_dict is not None and isinstance(stock_data_dict, dict):                                                       
    # Check available keys                                                                                                  
    available_keys = list(stock_data_dict.keys())                                                                           
    print(f"📊 Available data keys: {available_keys}")                                                                      
                                                                                                                            
    # Convert wide format dictionaries to long format DataFrame                                                             
    dfs_to_concat = []                                                                                                      
                                                                                                                            
    for price_type, wide_df in stock_data_dict.items():                                                                     
        if wide_df.empty:
            continue

        # Convert wide format to long format
        long_df = wide_df.reset_index().melt(
            id_vars='date',
            var_name='symbol',
            value_name=price_type.lower()
        )
        long_df = long_df.dropna(subset=[price_type.lower()])
        dfs_to_concat.append(long_df.set_index(['date', 'symbol']))

    if dfs_to_concat:
        # Combine all price types into single DataFrame
        df_daily = pd.concat(dfs_to_concat, axis=1).reset_index()

        # Rename columns to standard format
        column_mapping = { 
            'adjclose': 'adjclose',
            'close': 'close',
            'high': 'high',
            'low': 'low',
            'open': 'open',
            'volume': 'volume'
        }
        df_daily = df_daily.rename(columns=column_mapping)

        # Remove any rows with all NaN values
        df_daily = df_daily.dropna(how='all', subset=['open', 'high', 'low', 'close', 'adjclose', 'volume'])

        symbols = df_daily['symbol'].unique()
        date_range = f"{df_daily['date'].min()} to {df_daily['date'].max()}"

        print(f"✅ Converted to long format: {len(symbols)} symbols with {len(df_daily):,} total daily bars")
        print(f"📅 Date range: {date_range}")
        print(f"🔍 Sample symbols: {list(symbols[:10])}")
        print(f"📊 Columns: {list(df_daily.columns)}")

        # Check data quality
        missing_data = df_daily.isnull().sum().sum()
        if missing_data > 0:
            print(f"⚠️ Found {missing_data:,} missing values")
        else:
            print("✅ No missing values detected")
    else:
          print("❌ Failed to load stock data or unexpected format returned")
          df_daily = pd.DataFrame()

💾 stocks: loading cache /mnt/a61cc0e8-1b32-4574-a771-4ad77e8faab6/conda/technical_dashboard/cache/stock_data_universe.parquet
📊 Available data keys: ['AdjClose', 'Close', 'High', 'Low', 'Open', 'Volume']
✅ Converted to long format: 2953 symbols with 3,386,297 total daily bars
📅 Date range: 2020-09-21 00:00:00 to 2025-09-19 00:00:00
🔍 Sample symbols: ['A', 'AA', 'AACB', 'AACI', 'AACT', 'AAL', 'AAMI', 'AAOI', 'AAON', 'AAP']
📊 Columns: ['date', 'symbol', 'adjclose', 'close', 'high', 'low', 'open', 'volume']
⚠️ Found 270 missing values


In [4]:
def compute_weekly_stock_features(df_symbol: pd.DataFrame, symbol: str) -> pd.DataFrame:
    """
    Compute weekly resampled stock data with comprehensive technical features.
    
    Args:
        df_symbol: Daily OHLCV data for one symbol 
        symbol: Symbol name
        
    Returns:
        Weekly DataFrame with stock technical features (all prefixed with wk_)
    """
    if df_symbol.empty or 'date' not in df_symbol.columns:
        logger.warning(f"Empty or invalid data for {symbol}")
        return pd.DataFrame()
    
    try:
        # Step 1: Weekly resampling
        df_symbol = df_symbol.copy()
        df_symbol['date'] = pd.to_datetime(df_symbol['date'])
        df_weekly = df_symbol.set_index('date').copy()
        
        # Aggregation rules for OHLCV
        agg_rules = {
            'open': 'first',
            'high': 'max', 
            'low': 'min',
            'close': 'last',
            'adjclose': 'last',
            'volume': 'sum'
        }
        
        # Resample to weekly Friday bars
        weekly_data = df_weekly.resample('W-FRI').agg(agg_rules)
        weekly_data = weekly_data.dropna(subset=['close'])
        
        if weekly_data.empty or len(weekly_data) < 30:
            logger.warning(f"Insufficient weekly data for {symbol}: {len(weekly_data)} bars")
            return pd.DataFrame()
        
        # Add required columns
        weekly_data['week_end'] = weekly_data.index
        weekly_data['symbol'] = symbol
        
        # Calculate weekly returns
        src_col = 'adjclose' if 'adjclose' in weekly_data.columns else 'close'
        weekly_data['ret'] = np.log(pd.to_numeric(weekly_data[src_col], errors='coerce')).diff()
        
        weekly_data = weekly_data.reset_index(drop=True)
        
        # Store original columns to identify new features for prefixing
        original_cols = set(weekly_data.columns)
        
        # Step 2: Stock Technical Features
        
        # RSI features (6, 14, 21 week periods)
        weekly_data = add_rsi_features(
            weekly_data, 
            src_col=src_col, 
            periods=(6, 14, 21)
        )
        
        # MACD features (weekly: fast=5, slow=12, signal=4)
        weekly_data = add_macd_features(
            weekly_data,
            src_col=src_col,
            fast=5, slow=12, signal=4,
            derivative_ema_span=2
        )
        
        # Add 3-period EMA of MACD slope if MACD slope exists
        macd_slope_cols = [col for col in weekly_data.columns if 'macd_slope' in col]
        for macd_slope_col in macd_slope_cols:
            if not weekly_data[macd_slope_col].empty:
                ema3_col = f"{macd_slope_col}_ema3"
                weekly_data[ema3_col] = weekly_data[macd_slope_col].ewm(span=3, min_periods=2).mean()
        
        # Trend features
        weekly_data = add_trend_features(
            weekly_data,
            src_col=src_col,
            ma_periods=(20, 50),  
            slope_window=8,
            eps=1e-5
        )
        
        # Distance to MA features
        weekly_data = add_distance_to_ma_features(
            weekly_data,
            src_col=src_col,
            ma_lengths=(8, 21, 40),
            z_window=26
        )
        
        # Step 3: Advanced Features
        
        # Multiscale volatility regime features
        weekly_data = add_multiscale_vol_regime(weekly_data, src_col=src_col)
        
        # Hurst exponent features for trend persistence
        weekly_data = add_hurst_features(weekly_data, src_col=src_col)
        
        # Range breakout features
        weekly_data = add_range_breakout_features(weekly_data, src_col=src_col)
        
        # Volume features (if volume data available)
        if 'volume' in weekly_data.columns and not weekly_data['volume'].isna().all():
            weekly_data = add_volume_features(weekly_data, src_col=src_col)
            weekly_data = add_volume_shock_features(weekly_data, src_col=src_col)
        
        # Step 4: Add weekly prefix to all new features
        new_cols = set(weekly_data.columns) - original_cols
        feature_cols_to_rename = [col for col in new_cols 
                                 if col not in ['week_end', 'symbol', 'ret'] and not col.startswith('wk_')]
        
        rename_dict = {col: f"wk_{col}" for col in feature_cols_to_rename}
        weekly_data = weekly_data.rename(columns=rename_dict)
        
        logger.debug(f"Computed weekly stock features for {symbol}: {len(weekly_data)} rows, {len(rename_dict)} wk_ features")
        return weekly_data
        
    except Exception as e:
        logger.error(f"Error computing weekly stock features for {symbol}: {e}")
        return pd.DataFrame()


# Test the function with a small sample
print("🔧 Enhanced Weekly Stock Features Function Created (with wk_ prefix)")
print("   📊 Features computed:")
print("     - Weekly OHLCV resampling (Friday bars)")
print("     - wk_rsi_* (6, 14, 21 periods)")
print("     - wk_macd_* (5/12/4) + wk_macd_slope_ema3")
print("     - wk_trend_* and wk_ma_* (trend analysis)")
print("     - wk_dist_* (distance to moving averages)")
print("     - wk_vol_* (multiscale volatility regime)")
print("     - wk_hurst_* (trend persistence)")
print("     - wk_range_* (breakout features)")
print("     - wk_volume_* (volume and shock features)")

# Test with first few symbols if data is available
try:
    if 'df_daily' in locals() and not df_daily.empty:
        test_symbols = df_daily['symbol'].unique()[:2]
        print(f"\n🧪 Testing with {len(test_symbols)} symbols: {list(test_symbols)}")
        
        for test_symbol in test_symbols:
            test_data = df_daily[df_daily['symbol'] == test_symbol].copy()
            result = compute_weekly_stock_features(test_data, test_symbol)
            
            if not result.empty:
                wk_features = [col for col in result.columns if col.startswith('wk_')]
                print(f"   ✅ {test_symbol}: {len(result)} weekly bars, {len(wk_features)} wk_ features")
                print(f"      📋 Sample features: {wk_features[:5]}...")
            else:
                print(f"   ❌ {test_symbol}: Failed to generate features")
        
        print("\n✅ Enhanced weekly stock features function ready for parallel processing")
    else:
        print("\n⏳ Waiting for daily data to be loaded for testing")
except Exception as e:
    print(f"\n⚠️ Testing skipped: {e}")

🔧 Enhanced Weekly Stock Features Function Created (with wk_ prefix)
   📊 Features computed:
     - Weekly OHLCV resampling (Friday bars)
     - wk_rsi_* (6, 14, 21 periods)
     - wk_macd_* (5/12/4) + wk_macd_slope_ema3
     - wk_trend_* and wk_ma_* (trend analysis)
     - wk_dist_* (distance to moving averages)
     - wk_vol_* (multiscale volatility regime)
     - wk_hurst_* (trend persistence)
     - wk_range_* (breakout features)
     - wk_volume_* (volume and shock features)

🧪 Testing with 2 symbols: ['A', 'AA']


2025-10-18 11:43:26,579 - __main__ - ERROR - Error computing weekly stock features for A: add_multiscale_vol_regime() got an unexpected keyword argument 'src_col'
2025-10-18 11:43:26,790 - __main__ - ERROR - Error computing weekly stock features for AA: add_multiscale_vol_regime() got an unexpected keyword argument 'src_col'


   ❌ A: Failed to generate features
   ❌ AA: Failed to generate features

✅ Enhanced weekly stock features function ready for parallel processing


In [5]:
def compute_weekly_etf_features(df_etf: pd.DataFrame, symbol: str) -> pd.DataFrame:
    """
    Compute weekly resampled ETF data with technical features.
    
    Args:
        df_etf: Daily OHLCV data for one ETF symbol
        symbol: ETF symbol name
        
    Returns:
        Weekly DataFrame with ETF technical features
    """
    if df_etf.empty or 'date' not in df_etf.columns:
        logger.warning(f"Empty or invalid data for ETF {symbol}")
        return pd.DataFrame()
    
    try:
        # Step 1: Weekly resampling
        df_etf = df_etf.copy()
        df_etf['date'] = pd.to_datetime(df_etf['date'])
        df_weekly = df_etf.set_index('date').copy()
        
        # Aggregation rules for OHLCV
        agg_rules = {
            'open': 'first',
            'high': 'max', 
            'low': 'min',
            'close': 'last',
            'adjclose': 'last',
            'volume': 'sum'
        }
        
        # Resample to weekly Friday bars
        weekly_data = df_weekly.resample('W-FRI').agg(agg_rules)
        weekly_data = weekly_data.dropna(subset=['close'])
        
        if weekly_data.empty or len(weekly_data) < 30:
            logger.warning(f"Insufficient weekly data for ETF {symbol}: {len(weekly_data)} bars")
            return pd.DataFrame()
        
        # Add required columns
        weekly_data['week_end'] = weekly_data.index
        weekly_data['symbol'] = symbol
        
        # Calculate weekly returns
        src_col = 'adjclose' if 'adjclose' in weekly_data.columns else 'close'
        weekly_data['ret'] = np.log(pd.to_numeric(weekly_data[src_col], errors='coerce')).diff()
        
        weekly_data = weekly_data.reset_index(drop=True)
        
        # Step 2: ETF Technical Features
        
        # RSI features (6, 14, 21 week periods)
        weekly_data = add_rsi_features(
            weekly_data, 
            src_col=src_col, 
            periods=(6, 14, 21)
        )
        
        # MACD features (weekly: fast=5, slow=12, signal=4)
        weekly_data = add_macd_features(
            weekly_data,
            src_col=src_col,
            fast=5, slow=12, signal=4,
            derivative_ema_span=2
        )
        
        # Add 3-day EMA of MACD slope (critical for ETF relative strength analysis)
        macd_slope_cols = [col for col in weekly_data.columns if 'macd_slope' in col]
        for macd_slope_col in macd_slope_cols:
            if not weekly_data[macd_slope_col].empty:
                ema3_col = f"{macd_slope_col}_ema3"
                weekly_data[ema3_col] = weekly_data[macd_slope_col].ewm(span=3, min_periods=2).mean()
        
        logger.debug(f"Computed weekly ETF features for {symbol}: {len(weekly_data)} rows, {len(weekly_data.columns)} features")
        return weekly_data
        
    except Exception as e:
        logger.error(f"Error computing weekly ETF features for {symbol}: {e}")
        return pd.DataFrame()


# Test the function
print("🔧 Weekly ETF Features Function Created")
print("   📊 Features computed:")
print("     - Weekly OHLCV resampling (Friday bars)")
print("     - RSI (6, 14, 21 periods)")
print("     - MACD (5/12/4) + 3-day EMA of MACD slope")
print("     - Optimized for ETF benchmarking in relative strength")

# Test with ETF symbols if data is available
try:
    if 'df_daily' in locals() and not df_daily.empty:
        etf_symbols = [sym for sym in df_daily['symbol'].unique() 
                      if sym in ['SPY', 'RSP', 'XLK', 'XLF', 'XLE']][:3]
        
        if etf_symbols:
            print(f"\n🧪 Testing with {len(etf_symbols)} ETF symbols: {list(etf_symbols)}")
            
            for test_etf in etf_symbols:
                test_data = df_daily[df_daily['symbol'] == test_etf].copy()
                result = compute_weekly_etf_features(test_data, test_etf)
                
                if not result.empty:
                    technical_cols = [col for col in result.columns 
                                    if any(feat in col for feat in ['rsi_', 'macd_', 'ema3'])]
                    print(f"   ✅ {test_etf}: {len(result)} weekly bars, {len(technical_cols)} technical features")
                else:
                    print(f"   ❌ {test_etf}: Failed to generate features")
        else:
            print("\n⏳ No ETF symbols found in data for testing")
        
        print("\n✅ Step 3B completed: Weekly ETF features function ready for benchmarking")
    else:
        print("\n⏳ Waiting for daily data to be loaded for testing")
except Exception as e:
    print(f"\n⚠️ Testing skipped: {e}")

🔧 Weekly ETF Features Function Created
   📊 Features computed:
     - Weekly OHLCV resampling (Friday bars)
     - RSI (6, 14, 21 periods)
     - MACD (5/12/4) + 3-day EMA of MACD slope
     - Optimized for ETF benchmarking in relative strength

⏳ No ETF symbols found in data for testing

✅ Step 3B completed: Weekly ETF features function ready for benchmarking


## Step 3: Clean Weekly Stock Features Function

Let's create a focused function that computes weekly resampled stock data with technical features.